In [1]:
# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.axes_grid1 import make_axes_locatable

from qick import *
from qick.helpers import gauss

import logging
logger = logging.getLogger('qick.qick_asm')
logger.setLevel(logging.ERROR)

from tqdm import tqdm_notebook as tqdm

import time
import os
expt_path=os.getcwd()+'/data'
import sys
import scipy as sp
import json

import qutip as qt
import qutip.visualization as qplt

from slab.instruments import InstrumentManager
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 16})
# style.use('default')
style.use('S:\Connie\prx.mplstyle')

np.set_printoptions(threshold=sys.maxsize)

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


C:\ProgramData\Anaconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "


Could not load InstrumentManagerWindow
Could not load labbrick
Could not load Autonics TM4 Driver
Could not load Oxford Trition driver


In [3]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence

In [4]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

# Tomo analysis

In [5]:
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=3)

# Getting ideal rho

Define psi basis

In [6]:
# 2 cardinal points on Bloch sphere for each measurement axis
# psiZ = [qt.basis(3,0), qt.basis(3,1)]
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

In [7]:
def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)

    
def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    # print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

# Run experiment and analysis

In [8]:
"""Reopen saved data"""
def prev_data(expt_path, filename, verbose=True):
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    if verbose: print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        if verbose: print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [9]:
data_path = 'S:\\QRAM\\qram_4QR2\\'
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
expt_path = os.path.join(data_path, 'data', 'data_250119')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [10]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
# config_path = os.path.join(os.getcwd(), config_file)

# config_file = 'config.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)

print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


In [11]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [12]:
import Pyro4.util

## Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [13]:
# ns_address = '10.108.30.56' # Spilker Fermium
ns_address = '10.108.30.75' # Spilker Vibranium
im = InstrumentManager(ns_address=ns_address)
print(im.keys())

dict_keys(['Pyro.NameServer', 'Qick_QRAM'])


In [14]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [ ]:
lo1 = im[yaml_cfg.aliases.readout_LO]
lo1.open_device()
print(lo1.get_id())

# TURN OFF
lo1.set_standby(True)
lo1.set_output_state(False)

# # TURN ON
# lo1.set_standby(False)
# lo1.set_output_state(True)
# lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# lo_power = float(yaml_cfg.hw.lo.readout.power)
# lo1.set_frequency(lo_freq) # Set LO frequency
# print('set freq to', lo_freq)
# lo1.set_power(lo_power) # Set LO power
# print('set power to', lo_power)

## Run experiment and acquire data

### Test 3Q tomo

In [ ]:
save_data = True
saved_files = []

In [ ]:
tomo_qubits = [0, 2, 3] # which qubit/resonator we want to probe

In [ ]:
tomoExpt = meas.TestStateTomo3QExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"test_3q_state_tomo_{tomo_qubits[0]}{tomo_qubits[1]}{tomo_qubits[2]}",
    config_file=config_path,
)

tomoExpt.cfg.expt = dict(
    reps=20000, # reps per measurement basis
    singleshot_reps=30000, # reps for single shot calib
    tomo_qubits=tomo_qubits,
)

print(tomoExpt.cfg)
import Pyro4.util
try:
    tomoExpt.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

if save_data: saved_files.append(tomoExpt.save_data())

In [ ]:
tomo = tomoExpt
# tomo = qramExpt

# data, attrs = prev_data(expt_path, '00062_qram_protocol.h5')
# pulse_dict = attrs['pulse_dict']
# assert np.all(np.array(attrs['meas_order']) == meas_order)
# assert np.all(np.array(attrs['calib_order']) == calib_order)

print(meas_order)
print(calib_order)

data = tomo.data
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)
assert np.all(np.array(tomo.calib_order) == tomo_analysis.calib_order)

n_tomo_raw = np.array(data['counts_tomo'])
n_calib = np.array(data['counts_calib'])

np.set_printoptions(suppress=True)
print('Tomography counts')
print(n_tomo_raw)
print('Confusion matrix counts')
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib)))

### Qram experiment

In [536]:
tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
# tomo_qubits = [0, 1, 2] # which qubit/resonators to do tomo on

In [537]:
save_data = False
saved_files = []

In [585]:
# init_state = '|0>|0+i>' # Q0/Q1 (switch, input)
# init_state = '|1>|0+1>' # Q0/Q1 (switch, input)
# init_state = '|0+1>|1>' # Q0/Q1 (switch, input)
init_state = '|0+1>|0+1>' # Q0/Q1 (switch, input)

# init_state = 'Q2Q0_|1>|0>'
# init_state = '|0>|1>' # Q0/Q1 (switch, input)
# init_state = 'Q2Q0_|1>|0>'

# play_pulses = [0] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
play_pulses = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [1, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

# cool_qubits = [0, 1]
cool_qubits = None
readout_cool = False

full_mux_expt = True
resonator_reset = [0, 2, 3]

use_IQ_pulse = False

use_robust_pulses = True

sync_between_swaps = False
sync_between_efs = False

In [586]:
qramExpt = meas.QramProtocol3QTomoExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

# span = 2.161196963262976 # total time [us]
# span = 1.1659462011134218 # total time [us]
span = np.inf
npts = 1
# span = 1.8 # total time [us]
# npts = 120
# span = 0.8 # total time [us]
# npts = 10
if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))

qramExpt.cfg.expt = dict(
    # start=0, # start protocol time [us],
    # step=span/npts, # time step [us], 
    # expts=npts+1, # number of different time experiments - added 1 to include 0 step and full span

    start=span,
    step=0,
    expts=1, # number of different time experiments

    reps=1000000, # number of reps per tomo/time step,
    # reps=10000, # number of reps per tomo/time step,
    # reps=1000,
    # singleshot_reps=1, # reps per state for singleshot calibration
    # singleshot_reps=10000, # reps per state for singleshot calibration
    # singleshot_reps=1000, # reps per state for singleshot calibration
    singleshot_reps=10,
    post_process='scale', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
    tomo_3q=True, # flag to perform tomo on last time step
    tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
    init_state=init_state,
    play_pulses=play_pulses, # [2, 1, 4, 3]
    sync_between_swaps=sync_between_swaps,
    sync_between_efs=sync_between_efs,
    
    cool_qubits=cool_qubits,
    
    use_IQ_pulse=use_IQ_pulse,
    plot_IQ=False,
    
    readout_cool=readout_cool,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,
    
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    
    use_robust_pulses=use_robust_pulses,
    
    # meas_order=['XXX'],
)
if tomo_qubits == [0, 2, 3]:
    qramExpt.cfg.device.readout.gain[1] = 1e-4


# print(qramExpt.cfg)

import Pyro4.util
try:
    qramExpt.acquire(debug=False, progress=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# tomo = qramExpt
# data = tomo.data
# n_tomo_raw = np.array(data['counts_tomo'])
# n_calib = np.array(data['counts_calib'])

# n_tomo_raw_rounds.append(n_tomo_raw)
# n_calib_rounds.append(n_calib)
# n_tomo_rounds.append(fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib)))


timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 20.00000000000003 	 angle (deg): 0.0 	 threshold ge: 686.1283498556322
Qubit  (2)
ge fidelity (%): 29.999999999999993 	 angle (deg): 0.0 	 threshold ge: 377.00518722277997
Qubit  (3)
ge fidelity (%): 20.0 	 angle (deg): 0.0 	 threshold ge: 135.06780805119254
thresholds=[686.1283498556322, 0, 377.00518722277997, 135.06780805119254],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[-89.45622457282343, -54.142066720911316, 15.506590724166006, -140.57323026851097], [0, 0, 0, 0], [7.947843775427179, -1.3695687550854416, 41.16842961757527, 20.520097640358017], [-147.40504475183076, 85.90585842148089, 66.51952807160293, 143.88901545972334]] ,
counts_calib=[[0, 4, 0, 4, 0, 1, 0, 1], [0, 4, 1, 3, 0, 2, 0, 0], [1, 6, 0, 3, 0, 0, 0, 0], [0, 5, 1, 3, 0, 1, 0, 0], [0, 5, 1, 3, 0, 1, 0, 0], [1, 3, 1, 5, 0, 0, 0, 0], [0, 4, 1, 4, 0, 0, 0, 1], [1, 6, 0, 2, 0, 0, 0, 1]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [152]:
saved_files = [qramExpt.save_data()]

Saving S:\Connie\experiments\qramLL_4QR2\data\data_240617\00052_QramProtocol3QTomo.h5
S:\Connie\experiments\qramLL_4QR2\data\data_240617\00052_QramProtocol3QTomo.h5


#### post processing counts

In [361]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [362]:
tomo = qramExpt
data = tomo.data
cfg = tomo.cfg
pulse_dict = qramExpt.pulse_dict
init_state = qramExpt.cfg.expt.init_state

n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)

print('calib order', tomo_analysis.calib_order)

print('Raw tomography counts')
print(n_tomo_raw)
print('Confusion matrix counts')
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(correct_readout_err(n_tomo_raw, n_calib))) 

calib order ['ggg' 'gge' 'geg' 'gee' 'egg' 'ege' 'eeg' 'eee']
Raw tomography counts
[[2860  597 2010  123 2499 1335  392  184]
 [1461 2013 1100 1023  977 2885  181  360]
 [1693 1670 1115 1037 2674 1218  357  236]
 [1079  367 3703  442 1572  792 1292  753]
 [ 586  780 1956 2213  565 1869  506 1525]
 [ 713  689 2015 2056 1732  760 1359  676]
 [3419  403 1373  355 1570  735 1395  750]
 [1843 2079  740  996  522 1721  645 1454]
 [1902 1914  825  949 1549  761 1508  592]
 [4694 1155 1248  174  630  786 1190  123]
 [1980 3876  632  759  492  943  642  676]
 [3906 1957  748  625  670  733  707  654]
 [1890  680 3931  696  890  460 1012  441]
 [ 971 1675 1619 2996  338 1045  793  563]
 [1771  767 2868 1872  671  628  606  817]
 [3936  714 2053  641 1080  429  742  405]
 [1948 2598  808 1757  588 1036  649  616]
 [3038 1540 1554 1063  499 1077  828  401]
 [2429 1189 1076  173 2852  805 1334  142]
 [1583 2041  727  692  748 2696  641  872]
 [2487 1239  703  601 2007 1612  676  675]
 [ 871  642 2

In [139]:
filename = saved_files[-1]

IndexError: list index out of range

### Import optimal control pulse

In [ ]:
pulse_filepath = os.path.join(os.getcwd(), '240528_pulse_sequence_pp.npz')
pulse_params_dict = dict() # open file
with np.load(pulse_filepath) as npzfile:
    for key in npzfile.keys():
        pulse_params_dict.update({key:npzfile[key]})

print(pulse_params_dict.keys())
IQ_qubits = [0, 1]
print('IQ_qubits', IQ_qubits)

times = pulse_params_dict['times']
I_0 = pulse_params_dict['I_0']
Q_0 = pulse_params_dict['Q_0']
I_1 = pulse_params_dict['I_1']
Q_1 = pulse_params_dict['Q_1']
plt.figure()
plt.plot(times*1e6, I_0*1e-6, label='$I_0$')
plt.plot(times*1e6, Q_0*1e-6, label='$Q_0$')
plt.plot(times*1e6, I_1*1e-6, label='$I_1$')
plt.plot(times*1e6, Q_1*1e-6, label='$Q_1$')
plt.legend()
plt.xlabel('Times [us]')
plt.ylabel('Amplitude [MHz]')
plt.show()

I_values_MHz = np.array([I_0, I_1])*1e-6
Q_values_MHz = np.array([Q_0, Q_1])*1e-6
times_us = times*1e6

## Loop over delays on generators

In [419]:
init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']

protocol_play = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

sync_between_swaps = True
sync_between_efs = True

tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
save_data = True
saved_files = []
# cool_qubits = [0, 1]
cool_qubits = None

readout_cool = False
full_mux_expt = True
resonator_reset = [0, 2, 3]

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

delay_sweep_2_cycles = np.arange(4)
delay_sweep_3_cycles = np.arange(4)

delay_sweep_2 = []
for delay in delay_sweep_2_cycles:
    delay_sweep_2.append(soc.cycles2us(delay)*1e3)
delay_sweep_3 = []
for delay in delay_sweep_3_cycles:
    delay_sweep_3.append(soc.cycles2us(delay)*1e3)
print("delay_sweep_2", delay_sweep_2)
print("delay_sweep_3", delay_sweep_3)

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

delay_sweep_2 [0.0, 2.3251488095238093, 4.650297619047619, 6.975446428571429]
delay_sweep_3 [0.0, 2.3251488095238093, 4.650297619047619, 6.975446428571429]
init_states (4 states) ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
protocol_play [2, 1, 4, 3]
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt True
resonator reset [0, 2, 3]
use robust pulses True
save data True


In [424]:
play_pulses = protocol_play
for delay_2 in delay_sweep_2:
    for delay_3 in delay_sweep_3:
        print('delays', delay_2, delay_3)
        for init_state in init_states:
            qramExpt = meas.QramProtocol3QTomoExperiment(
                soccfg=soc,
                path=expt_path,
                config_file=config_path,
                ns_address=ns_address,
            )
            print('init_state', init_state, 'play pulses', play_pulses)
            
            span = np.inf
            npts = 1
            if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
            
            qramExpt.cfg.expt = dict(
                start=span,
                step=0,
                expts=1, # number of different time experiments
            
                reps=7500, # number of reps per tomo/time step,
                singleshot_reps=7500, # reps per state for singleshot calibration
                # reps=100, # number of reps per tomo/time step,
                # singleshot_reps=100, #000, # reps per state for singleshot calibration
                post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
                tomo_3q=True, # flag to perform tomo on last time step
                tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
                init_state=init_state,
                play_pulses=play_pulses, # [2, 1, 4, 3]
                cool_qubits=cool_qubits,
                sync_between_swaps=sync_between_swaps,
                sync_between_efs=sync_between_efs,
                
                use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
                use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
                plot_IQ=False,
                
                readout_cool=readout_cool,
                n_init_readout=1,
                n_trig=1,
                init_read_wait_us=10,
                
                full_mux_expt=full_mux_expt,
                resonator_reset=resonator_reset,
            )
            if tomo_qubits == [0, 2, 3]:
                qramExpt.cfg.device.readout.gain[1] = 1e-4
                
            qramExpt.cfg.hw.soc.dacs.delay_chs.delay_ns[2] = delay_2
            qramExpt.cfg.hw.soc.dacs.delay_chs.delay_ns[3] = delay_3

            
            try:
                qramExpt.acquire(progress=True, debug=False)
                success = True
            except Exception:
                print("Pyro traceback:")
                print("".join(Pyro4.util.getPyroTraceback()))
                
            if save_data: saved_files.append(qramExpt.save_data())

delays 0.0 0.0
init_state |0+1>|1> play pulses [2, 1, 4, 3]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 75.45145041561096 	 angle (deg): 0.0 	 threshold ge: 999.7444509165722
Qubit  (2)
ge fidelity (%): 78.47999999999998 	 angle (deg): 0.0 	 threshold ge: 902.2436566442781
Qubit  (3)
ge fidelity (%): 75.90517313419566 	 angle (deg): 0.0 	 threshold ge: 813.0000825875447
thresholds=[999.7444509165722, 0, 902.2436566442781, 813.0000825875447],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[0.40442712232167083, 3.795851695145104, 0.30697618660157217, -2.154130729590451], [0, 0, 0, 0], [2.5030405207485753, -4.534565988608625, 5.922412367778682, -0.9271806889069698], [-0.5853493897477616, -4.273845945212909, -0.14435215622457592, -0.2123881746677449]] ,
counts_calib=[[5148, 649, 537, 78, 850, 111, 109, 18], [808, 5362, 68, 551, 66, 586, 9, 50], [673, 91, 5256, 723, 98, 14, 576, 69], [86, 595, 845, 5217, 8, 71, 90, 588], [624, 61, 64, 9, 5349, 672, 638, 83], [115, 587, 10, 55, 1021, 5013, 96, 603], [84, 7, 634, 72, 671, 76, 5276, 680], [34, 118, 112, 663, 146, 747, 799, 4881]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 80.93333333333331 	 angle (deg): 0.0 	 threshold ge: 990.0118222728438
Qubit  (2)
ge fidelity (%): 79.67872160732543 	 angle (deg): 0.0 	 threshold ge: 865.1884132480717
Qubit  (3)
ge fidelity (%): 75.77333333333335 	 angle (deg): 0.0 	 threshold ge: 770.6890200062034
thresholds=[990.0118222728438, 0, 865.1884132480717, 770.6890200062034],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[2.093766856522919, 0.9666214266341199, 3.118946894494168, 1.1174743694060219], [0, 0, 0, 0], [-0.29963786276105386, -0.28025071874152346, 5.150564361269323, -1.1513325739083253], [-6.917106265256305, 7.183677895307838, -6.252177163005149, 2.494957851912124]] ,
counts_calib=[[5364, 750, 600, 95, 534, 76, 73, 8], [732, 5245, 85, 692, 66, 582, 9, 89], [622, 74, 5495, 665, 58, 4, 521, 61], [95, 622, 856, 5246, 8, 72, 95, 506], [598, 68, 71, 8, 5238, 685, 730, 102], [100, 650, 9, 79, 780, 5012, 103, 767], [82, 7, 648, 66, 634, 91, 5229, 743], [20, 84, 109, 739, 71, 647, 786, 5044]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_241025\00658_QramProtocol3QTomo.h5


In [425]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00595_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00596_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00597_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00598_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00599_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00600_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00601_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00602_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00603_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00604_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00605_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00606_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00607_QramProtocol3QTomo.h5', 'S:\\QRAM\\

In [444]:
n_tomo_corrected_dict = dict()

all_configs = dict()
i_saved_file = 0
for i_delay_2, delay_2 in enumerate(tqdm(delay_sweep_2)):
    for i_delay_3, delay_3 in enumerate(tqdm(delay_sweep_3)):
        for init_state in init_states:
            saved_file = saved_files[i_saved_file]
            print(saved_file)
            data, attrs = prev_data(saved_file, None, verbose=False)
            pulse_dict = attrs['pulse_dict']
            cfg = AttrDict(attrs['config'])
            assert cfg.expt.init_state == init_state
            all_configs[init_state] = cfg
            play_pulses = cfg.expt.play_pulses
            print('init state Q0 (switch) Q1 (input):', init_state)
            
            n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
            n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
            
            if init_state not in n_tomo_corrected_dict.keys():
                n_tomo_corrected_dict.update({init_state:np.zeros((len(delay_sweep_2), len(delay_sweep_3), 27, 8))})
            
            n_tomo_corrected_dict[init_state][i_delay_2, i_delay_3] = n_tomo_corrected
            i_saved_file += 1

init_states = np.copy(list(n_tomo_corrected_dict.keys()))

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\1847658743.py: 5

  0%|          | 0/4 [00:00<?, ?it/s]

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\1847658743.py: 6

  0%|          | 0/4 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\00595_QramProtocol3QTomo.h5


...

S:\QRAM\qram_4QR2\data\data_241025\00611_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00612_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00613_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00614_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00615_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00616_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00617_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00618_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00619_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_24102

  0%|          | 0/4 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\00627_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00628_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00629_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00630_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00631_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00632_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00633_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00634_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00635_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_24102

  0%|          | 0/4 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\00643_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00644_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00645_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00646_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00647_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_241025\00648_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00649_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00650_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00651_QramProtocol3QTomo.h5
init state Q0 (switch) Q1 (input): |0+1>|1>
S:\QRAM\qram_4QR2\data\data_24102

In [445]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})
n_tomo_corrected_dict.update({"delay_sweep_2":delay_sweep_2})
n_tomo_corrected_dict.update({"delay_sweep_3":delay_sweep_3})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412052137_ntomocorrected_3Q_023.npz


In [428]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
# rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202411051014_init_rhoMLE_ZZ_2Q_01.npz'
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412051654_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_241025\202412051654_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|1>|0+1>', '|1>|0+i>', '|1>|1>', '|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


In [429]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

Reopening 3q tomo data

In [518]:
# RETRIEVING MEASURED n_tomo_corrected
n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412052137_ntomocorrected_3Q_023.npz"
print(f"Using n_tomo_corrected_dict from file {n_tomo_corrected_filepath}")
n_tomo_corrected_dict = dict()

n_tomo_corrected_dict.update({"saved_files":saved_files})
with np.load(n_tomo_corrected_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        n_tomo_corrected_dict.update({key:npzfile[key]})

init_states = n_tomo_corrected_dict["init_states"]

print(f'n_tomo_corrected_dict retrieved with init_states\n{init_states}')
all_configs = n_tomo_corrected_dict["all_configs"].item() #[0]
# print(all_configs.keys())


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Using n_tomo_corrected_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412052137_ntomocorrected_3Q_023.npz
n_tomo_corrected_dict retrieved with init_states
['|0+1>|1>' '|1>|0+1>' '|0>|0+1>' '|0+1>|0+1>']
play_pulses [2, 1, 4, 3]


Looping MLE

In [519]:
use_init_2q_state = True

In [520]:
rhos_base = []
fidelities_base = []
purities_base = []

rhos_ZZ = []
fidelities_ZZ = []
purities_ZZ = []

In [521]:
for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected_delays = n_tomo_corrected_dict[init_state]
    print(n_tomo_corrected_delays.shape)
    
    for i_delay_2, delay_2 in enumerate(delay_sweep_2):
        for i_delay_3, delay_3 in enumerate(delay_sweep_3):
            n_tomo_corrected = n_tomo_corrected_delays[i_delay_2, i_delay_3]
            
            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
            rho_id = get_qram_qSLR_state_from_rho(init_rho)
            
            # ------------------------------ #
            # Base MLE
            # ------------------------------ #
            
            rho_MLE_base = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=False,
            )
         
            fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
            purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
            purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
            
            rhos_base.append(rho_MLE_base)
            fidelities_base.append(fid_base)
            purities_base.append(purity_base)
            
            rho_MLE = rho_MLE_base
        
            # ------------------------------ #
            # MLE with ZZ
            # ------------------------------ #
        
            rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=True,
                evol_mats=evol_mats
            )
        
            fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
            purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
            purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
        
            rhos_ZZ.append(rho_MLE_ZZ)
            fidelities_ZZ.append(fid_ZZ)
            purities_ZZ.append(purity_ZZ)

init state (Q0 (switch) Q1 (input)): |0+1>|1>
(4, 4, 27, 8)
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |1>|0+1>
(4, 4, 27, 8)
constructing final state on |switch, out1, out2>
constructing final state on |switch, out1, out2

Run loop over virtual Z rotations

In [522]:
use_ZZ_correction = True
show_fids_grid = False

In [523]:
fidelities_opt_rot = []
all_best_phis = []
saveplots = False
# save_append = '_compareMeas'
save_append = ''
phis = [None, None, None]

In [524]:
phis[0] = np.linspace(0, 360, 19) # Q0
phis[1] = np.linspace(0, 360, 19) # Q2
phis[2] = np.linspace(0, 360, 19) # Q3

print(phis)

[array([  0.,  20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200.,
       220., 240., 260., 280., 300., 320., 340., 360.]), array([  0.,  20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200.,
       220., 240., 260., 280., 300., 320., 340., 360.]), array([  0.,  20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200.,
       220., 240., 260., 280., 300., 320., 340., 360.])]


In [525]:
fids_grids_all_delays = np.zeros((len(init_states), len(delay_sweep_2), len(delay_sweep_3), len(phis[0]), len(phis[1]), len(phis[2])))
for i_init_state, init_state in enumerate(init_states):
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected_delays = n_tomo_corrected_dict[init_state]
    
    for i_delay_2, delay_2 in enumerate(delay_sweep_2):
        for i_delay_3, delay_3 in enumerate(delay_sweep_3):
            n_tomo_corrected = n_tomo_corrected_delays[i_delay_2, i_delay_3]
            
            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
            rho_id = get_qram_qSLR_state_from_rho(init_rho)
            
            rho_MLE = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=True,
                method='analytical',
                ZZ_correction=use_ZZ_correction,
                evol_mats=None if not use_ZZ_correction else evol_mats,
            )
            
            rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis, progress=True)
            fids_grids_all_delays[i_init_state, i_delay_2, i_delay_3] = fids_grid
            print("best_phis", best_phis)
            print("best_fid", best_fid)

init state (Q0 (switch) Q1 (input)): |0+1>|1>
constructing final state on |switch, out1, out2>


  0%|          | 0/6859 [00:00<?, ?it/s]

Improved fidelity by (%) 69.57161952823967
best_phis [ 80.  60. 220.]
best_fid 0.9464743541936589
constructing final state on |switch, out1, out2>


  0%|          | 0/6859 [00:00<?, ?it/s]

Improved fidelity by (%) 54.35342797186578
best_phis [120. 240. 220.]
best_fid 0.9678147868310295
constructing final state on |switch, out1, out2>


...

Improved fidelity by (%) 30.23695576267863
best_phis [  0.  40. 100.]
best_fid 0.9200749014995881
constructing final state on |switch, out1, out2>


  0%|          | 0/6859 [00:00<?, ?it/s]

Improved fidelity by (%) 36.92385017388754
best_phis [  0. 260. 320.]
best_fid 0.9682006700211586
constructing final state on |switch, out1, out2>


  0%|          | 0/6859 [00:00<?, ?it/s]

Improved fidelity by (%) 22.652478213681214
best_phis [  0.  40. 300.]
best_fid 0.9378132935307635


Go through the fids grids and select the best overall phi

In [527]:
check_init_states = init_states
# check_init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
# check_init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+i>']

In [529]:
fids_grids_all_delays.shape

(4, 4, 4, 19, 19, 19)

In [530]:
best_avg_fids_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3)))
best_phi_indices_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), 3))
best_phis_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), 3))
best_fids_all_delays = np.zeros((len(delay_sweep_2), len(delay_sweep_3), len(check_init_states)))

# Pick best overall phi for each delay set
for i_delay_2, delay_2 in enumerate(delay_sweep_2):
    for i_delay_3, delay_3 in enumerate(delay_sweep_3):
        best_avg_fid = 0
        best_phis = [0, 0, 0]
        best_phi_indices = [0, 0, 0]
        for iphi0, phi0 in enumerate(phis[0]):
            for iphi1, phi1 in enumerate(phis[1]):
                for iphi2, phi2 in enumerate(phis[2]):
                    avg_fid = 0
                    for i_state, init_state in enumerate(init_states):
                        if init_state not in check_init_states: continue
                        avg_fid += fids_grids_all_delays[i_state, i_delay_2, i_delay_3, iphi0, iphi1, iphi2]
                    avg_fid /= len(check_init_states)
                    if avg_fid > best_avg_fid:
                        best_phis = [phi0, phi1, phi2]
                        best_phi_indices = [iphi0, iphi1, iphi2]
                        best_avg_fid = avg_fid
                        i_check_init_state = 0
                        for i_state, init_state in enumerate(init_states):
                            if init_state not in check_init_states: continue
                            best_fids_all_delays[i_delay_2, i_delay_3, i_check_init_state] = fids_grids_all_delays[i_state, i_delay_2, i_delay_3, iphi0, iphi1, iphi2]
                            i_check_init_state += 1
        best_avg_fids_all_delays[i_delay_2, i_delay_3] = best_avg_fid
        best_phi_indices_all_delays[i_delay_2, i_delay_3] = best_phi_indices
        best_phis_all_delays[i_delay_2, i_delay_3] = best_phis

# Pick best delay set
best_delay_indices = np.unravel_index(np.argmax(best_avg_fids_all_delays), best_avg_fids_all_delays.shape)
best_delays = [delay_sweep_2[best_delay_indices[0]], delay_sweep_3[best_delay_indices[1]]]
best_avg_fid = best_avg_fids_all_delays[best_delay_indices]
best_phi_indices = best_phi_indices_all_delays[best_delay_indices]
best_phis = best_phis_all_delays[best_delay_indices]
best_fids = best_fids_all_delays[best_delay_indices]

print(init_states)
print('best delays', best_delays)
print('best avg fid', best_avg_fid)
print('best fids for check states', best_fids)
print('best phis', best_phis)

['|0+1>|1>' '|1>|0+1>' '|0>|0+1>' '|0+1>|0+1>']
best delays [4.650297619047619, 6.975446428571429]
best avg fid 0.9261171042442358
best fids for check states [0.94152904 0.93084578 0.96843406 0.86365953]
best phis [360.  40. 140.]


In [535]:
outer_sweep = delay_sweep_2
inner_sweep = delay_sweep_3
y_sweep = outer_sweep
x_sweep = inner_sweep

plt.figure()
plt.title(f'Avg Fidelities for Optimal Phis')
plt.xlabel(f'Delays Ch 3 (ns)')
plt.ylabel(f'Delays Ch 2 (ns)')
plt.pcolormesh(x_sweep, y_sweep, best_avg_fids_all_delays, cmap='viridis', shading='auto')
plt.colorbar()
plt.clim(vmin=0.8, vmax=vmax)
plt.show()

In [ ]:
inner_sweep = phis[2]
outer_sweep = phis[1]
y_sweep = outer_sweep
x_sweep = inner_sweep
print(fids_grid_all_delays[i_state, best_delay_indices[0], best_delay_indices[1], i_phi0].shape)
print(f'phi Q0 {phi0}')
plt.figure()
plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})')
plt.xlabel(f'phis Q{tomo_qubits[2]}')
plt.ylabel(f'phis Q{tomo_qubits[1]}')
plt.pcolormesh(x_sweep, y_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
plt.colorbar()
plt.clim(vmin=0, vmax=vmax)
plt.show()

## Big qram protocol loop!

In [15]:
cardinal_states = ['|0>', '|1>', '|0+1>', '|0+i>']
init_states = []
for state0 in cardinal_states:
    for state1 in cardinal_states:
        init_states.append(state0 + state1)

# init_states = ['|0+1>|0>', '|0>|0+1>', '|1>|0+1>', '|0+1>|1>', '|0+1>|0+1>']
# init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
# init_states = ['|0+1>|0+1>']

protocol_play = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

sync_between_swaps = True
sync_between_efs = True

tomo_qubits = [0, 2, 3] # which qubit/resonators to do tomo on
save_data = True
saved_files = []
# cool_qubits = [0, 1]
cool_qubits = None

readout_cool = False
full_mux_expt = True
resonator_reset = [0, 2, 3]

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

iterations = 5
# iterations = 1

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 4, 3]
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt True
resonator reset [0, 2, 3]
use robust pulses True
save data True


In [666]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

init_state |0>|0> play pulses [2, 1, 4, 3]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 79.29921292129217 	 angle (deg): 0.0 	 threshold ge: 1002.2141649298954
Qubit  (2)
ge fidelity (%): 78.59000000000002 	 angle (deg): 0.0 	 threshold ge: 905.4298295308482
Qubit  (3)
ge fidelity (%): 82.10000000000002 	 angle (deg): 0.0 	 threshold ge: 786.9301992147205
thresholds=[1002.2141649298954, 0, 905.4298295308482, 786.9301992147205],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[-0.798294385679414, 2.239152481692433, 2.141598291293734, 1.876731651749389], [0, 0, 0, 0], [2.1800183075671273, -2.6088668022782744, 2.9974648494711142, 0.7431022782750198], [0.3322484133441831, -1.532851505288854, 2.973341741253051, 7.6877056143205875]] ,
counts_calib=[[7761, 615, 731, 69, 695, 54, 69, 6], [876, 7436, 74, 726, 101, 712, 8, 67], [1085, 87, 7388, 585, 97, 10, 689, 59], [145, 950, 812, 7074, 15, 121, 96, 787], [1082, 88, 73, 8, 7474, 572, 645, 58], [146, 1085, 10, 86, 919, 7020, 68, 666], [165, 12, 986, 90, 971, 80, 7148, 548], [15, 134, 115, 982, 101, 880, 729, 7044]]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.90000000000003 	 angle (deg): 0.0 	 threshold ge: 974.9880944824819
Qubit  (2)
ge fidelity (%): 79.13913391339136 	 angle (deg): 0.0 	 threshold ge: 893.1740133323506
Qubit  (3)
ge fidelity (%): 80.12925192519255 	 angle (deg): 0.0 	 threshold ge: 799.8029428822366
thresholds=[974.9880944824819, 0, 893.1740133323506, 799.8029428822366],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[-0.692058502847845, -3.0689256305939794, -1.7289739625711957, 2.400700162733931], [0, 0, 0, 0], [-1.8335471928397062, 1.8943346623270958, 2.310363303498779, 2.4846550040683493], [1.4197129373474366, 4.216077786818551, -1.113147681041496, -0.25609585028478177]] ,
counts_calib=[[7554, 634, 765, 69, 813, 66, 91, 8], [995, 7165, 92, 746, 107, 800, 16, 79], [951, 92, 7223, 692, 114, 11, 851, 66], [174, 917, 983, 6887, 16, 113, 127, 783], [930, 86, 120, 13, 7268, 748, 758, 77], [122, 919, 12, 94, 916, 6989, 105, 843], [133, 10, 918, 93, 892, 92, 7156, 706], [18, 114, 122, 946, 114, 845, 838, 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_241025\00859_QramProtocol3QTomo.h5


In [667]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00780_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00781_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00782_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00783_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00784_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00785_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00786_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00787_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00788_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00789_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00790_QramProtocol3QTomo.h5', 'S:\\QRAM\\

#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [668]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\3499015544.py: 3

  0%|          | 0/80 [00:00<?, ?it/s]

 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\1887187011.py: 13

S:\QRAM\qram_4QR2\data\data_241025\00780_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00780_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0>
S:\QRAM\qram_4QR2\data\data_241025\00781_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00781_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_241025\00782_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00782_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00783_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00783_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+i>
S:\QRAM\

In [669]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412081553_ntomocorrected_3Q_023.npz


### Round 1a with simultaneous pulses

In [290]:
# saved_files = ['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00515_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00516_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00517_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00518_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00519_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00520_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00521_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00522_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00523_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00524_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00525_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00526_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00527_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00528_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00529_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00530_QramProtocol3QTomo.h5']
# iterations = 4

In [291]:
# saved_files = []

sync_between_swaps = False
sync_between_efs = False

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 4, 3]
simultaneous swaps True
simultaneous efs True
readout cool False
full mux expt True
resonator reset [0, 2, 3]
use robust pulses True
save data True


In [292]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

init_state |0>|0> play pulses [2, 1, 4, 3]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 70.5386708670867 	 angle (deg): 0.0 	 threshold ge: 591.8836790703319
Qubit  (2)
ge fidelity (%): 78.63907290729071 	 angle (deg): 0.0 	 threshold ge: 900.2559942556144
Qubit  (3)
ge fidelity (%): 75.13876087608767 	 angle (deg): 0.0 	 threshold ge: 804.1872560517737
thresholds=[591.8836790703319, 0, 900.2559942556144, 804.1872560517737],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[-0.2818724165988609, 2.638253376729048, 3.66933897477624, 0.21111668022782723], [0, 0, 0, 0], [-0.46168096013018933, -4.568513995117983, -3.9084514239218864, 0.14147379983726588], [3.996989503661514, -2.5810567127746142, 0.31246647681041795, 4.430258665581778]] ,
counts_calib=[[6777, 959, 727, 137, 1112, 162, 113, 13], [955, 6793, 91, 781, 159, 1092, 13, 116], [874, 119, 6735, 947, 138, 16, 1043, 128], [280, 1039, 1104, 6256, 37, 182, 180, 922], [1204, 185, 123, 34, 6641, 946, 735, 132], [227, 1183, 26, 128, 1167, 6377, 133, 759], [191, 32, 1195, 189, 878, 143, 6429, 943], [59, 459, 166,

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 68.75000000000001 	 angle (deg): 0.0 	 threshold ge: 571.0969499328503
Qubit  (2)
ge fidelity (%): 80.57000000000001 	 angle (deg): 0.0 	 threshold ge: 875.3165688569608
Qubit  (3)
ge fidelity (%): 73.44867786778676 	 angle (deg): 0.0 	 threshold ge: 776.4388903948715
thresholds=[571.0969499328503, 0, 875.3165688569608, 776.4388903948715],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[4.612658421480878, 1.6984442636289667, -4.9033571196094385, -0.3600078925956052], [0, 0, 0, 0], [1.9133109031733118, 2.072791944670463, -4.007023596419853, 5.516621318144834], [-2.8947919446704633, -6.384667290480065, 2.057535720097645, 2.4175171684296175]] ,
counts_calib=[[6561, 983, 703, 134, 1239, 209, 141, 30], [956, 6510, 107, 803, 215, 1197, 26, 186], [677, 114, 6560, 1050, 130, 21, 1272, 176], [144, 808, 1174, 6339, 40, 166, 204, 1125], [1177, 187, 127, 18, 6396, 1146, 802, 147], [215, 1104, 27, 147, 1139, 6398, 129, 841], [155, 24, 1116, 203, 728, 117, 6531, 1126], [25, 175, 183

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_241025\00594_QramProtocol3QTomo.h5


In [293]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00515_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00516_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00517_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00518_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00519_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00520_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00521_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00522_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00523_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00524_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00525_QramProtocol3QTomo.h5', 'S:\\QRAM\\

#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [294]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\2863891231.py: 3

  0%|          | 0/80 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\00515_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00515_QramProtocol3QTomo.h5


 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\809401443.py: 13

Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0>
S:\QRAM\qram_4QR2\data\data_241025\00516_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00516_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_241025\00517_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00517_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00518_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00518_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+i>
S:\QRAM\qram_4QR2\data\data_241025\00519_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00519_QramProtocol3QTomo.h5
Extracte

NameError: name 'copy' is not defined

In [296]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412051123_ntomocorrected_3Q_023.npz


### Round 2 with 3 swap applications

In [670]:
saved_files = []

init_states = []
for state0 in cardinal_states:
    for state1 in cardinal_states:
        init_states.append(state0 + state1)

sync_between_swaps = True
sync_between_efs = True

protocol_play = [2, 1, 1, 2, 2, 1, 4, 3]

iterations = 5

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 1, 2, 2, 1, 4, 3]
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt True
resonator reset [0, 2, 3]
use robust pulses True
save data True


In [671]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

init_state |0>|0> play pulses [2, 1, 1, 2, 2, 1, 4, 3]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.58909990999102 	 angle (deg): 0.0 	 threshold ge: 984.5851552535489
Qubit  (2)
ge fidelity (%): 79.5391799179918 	 angle (deg): 0.0 	 threshold ge: 921.0183123544316
Qubit  (3)
ge fidelity (%): 79.46999999999994 	 angle (deg): 0.0 	 threshold ge: 806.2711620491621
thresholds=[984.5851552535489, 0, 921.0183123544316, 806.2711620491621],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[10.336067615947925, 0.9964254678600492, 5.113285842148088, -2.605001952807161], [0, 0, 0, 0], [-0.2969720097640362, -0.2662471928397067, 0.28467624084621623, -2.7874104963384867], [-0.705316761594792, -1.9533407648494707, 0.9264039869812788, 1.3300229454841375]] ,
counts_calib=[[7631, 623, 743, 67, 790, 73, 65, 8], [1072, 7134, 89, 670, 112, 838, 15, 70], [972, 80, 7325, 655, 105, 9, 790, 64], [114, 756, 981, 7163, 13, 82, 107, 784], [1035, 74, 90, 9, 7321, 643, 748, 80], [146, 1004, 14, 100, 958, 6990, 93, 695], [149, 12, 1013, 88, 932, 80, 7231, 495], [23, 158, 128, 936, 157, 1008, 729

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.77999999999999 	 angle (deg): 0.0 	 threshold ge: 985.9873186633381
Qubit  (2)
ge fidelity (%): 79.97 	 angle (deg): 0.0 	 threshold ge: 885.793013120136
Qubit  (3)
ge fidelity (%): 80.66000000000003 	 angle (deg): 0.0 	 threshold ge: 786.2721014330116
thresholds=[985.9873186633381, 0, 885.793013120136, 786.2721014330116],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[[2.9792657445077313, 6.629816598860864, 9.724383075671277, -1.1365575264442638], [0, 0, 0, 0], [3.503286574450772, 2.227309194467048, -2.6991340927583414, -1.4841700569568759], [-3.9790030919446706, 2.866153132628154, -2.200485679414155, 1.4669224572823392]] ,
counts_calib=[[7457, 636, 840, 76, 787, 89, 100, 15], [914, 7068, 105, 881, 97, 803, 14, 118], [817, 77, 7498, 680, 83, 8, 775, 62], [99, 743, 954, 7194, 10, 88, 133, 779], [924, 104, 101, 7, 7163, 676, 925, 100], [143, 965, 21, 142, 926, 6765, 119, 919], [130, 14, 967, 107, 804, 90, 7166, 722], [28, 125, 138, 971, 99, 846, 979, 6814]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_241025\00939_QramProtocol3QTomo.h5


In [672]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00860_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00861_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00862_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00863_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00864_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00865_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00866_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00867_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00868_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00869_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00870_QramProtocol3QTomo.h5', 'S:\\QRAM\\

#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [673]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\3499015544.py: 3

  0%|          | 0/80 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\00860_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00860_QramProtocol3QTomo.h5


 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\1887187011.py: 13

Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0>
S:\QRAM\qram_4QR2\data\data_241025\00861_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00861_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_241025\00862_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00862_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\00863_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00863_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+i>
S:\QRAM\qram_4QR2\data\data_241025\00864_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\00864_QramProtocol3QTomo.h5
Extracte

In [674]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412082023_ntomocorrected_3Q_023.npz


### Round 3 with 5 swap applications

In [16]:
saved_files = []

# init_states = ['|0>|1>', '|1>|1>', '|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
init_states = []
for state0 in cardinal_states:
    for state1 in cardinal_states:
        init_states.append(state0 + state1)

sync_between_swaps = True
sync_between_efs = True

# protocol_play = [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]
protocol_play = [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]

iterations = 5

print(f'init_states ({len(init_states)} states)', init_states)
print('protocol_play', protocol_play)
print('simultaneous swaps', not sync_between_swaps)
print('simultaneous efs', not sync_between_efs)
print('readout cool', readout_cool)
print('full mux expt', full_mux_expt)
print('resonator reset', resonator_reset)
print('use robust pulses', use_robust_pulses)
print('save data', save_data)
if not save_data:
    print('WARNING!! YOU ARE NOT SAVING DATA!')

init_states (16 states) ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
protocol_play [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]
simultaneous swaps False
simultaneous efs False
readout cool False
full mux expt True
resonator reset [0, 2, 3]
use robust pulses True
save data True


In [17]:
play_pulses = protocol_play
for it in range(iterations):
    for init_state in init_states:
        qramExpt = meas.QramProtocol3QTomoExperiment(
            soccfg=soc,
            path=expt_path,
            config_file=config_path,
            ns_address=ns_address,
        )
        print('init_state', init_state, 'play pulses', play_pulses)
        
        span = np.inf
        npts = 1
        if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
        
        qramExpt.cfg.expt = dict(
            start=span,
            step=0,
            expts=1, # number of different time experiments
        
            reps=10000, # number of reps per tomo/time step,
            singleshot_reps=10000, # reps per state for singleshot calibration
            # reps=100, # number of reps per tomo/time step,
            # singleshot_reps=100, #000, # reps per state for singleshot calibration
            post_process=None, # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
            tomo_3q=True, # flag to perform tomo on last time step
            tomo_qubits=tomo_qubits, # the qubits on which to do the 3q state tomo
            init_state=init_state,
            play_pulses=play_pulses, # [2, 1, 4, 3]
            cool_qubits=cool_qubits,
            sync_between_swaps=sync_between_swaps,
            sync_between_efs=sync_between_efs,
            
            use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
            use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
            plot_IQ=False,
            
            readout_cool=readout_cool,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=10,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
        if tomo_qubits == [0, 2, 3]:
            qramExpt.cfg.device.readout.gain[1] = 1e-4
        
        try:
            qramExpt.acquire(progress=True, debug=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
            
        if save_data: saved_files.append(qramExpt.save_data())

init_state |0>|0> play pulses [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]
timesteps [inf]


  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.99999999999999 	 angle (deg): 0.0 	 threshold ge: 979.0777301920277
Qubit  (2)
ge fidelity (%): 81.47999999999998 	 angle (deg): 0.0 	 threshold ge: 858.4021706253319
Qubit  (3)
ge fidelity (%): 72.17 	 angle (deg): 0.0 	 threshold ge: 805.0207063755438
thresholds=[979.0777301920277, 0, 858.4021706253319, 805.0207063755438],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[array([ 592.66515905,    0.        , 1408.8113844 ,    0.        ]), [0, 0, 0, 0], array([ 515.61017803,    0.        , 1306.99489403,    0.        ]), array([ 481.72032206,    0.        , 1255.95044008,    0.        ])] ,
counts_calib=[[7125, 1050, 721, 145, 750, 104, 93, 12], [1200, 6806, 140, 860, 121, 768, 17, 88], [721, 89, 7209, 1014, 74, 13, 758, 122], [164, 752, 1249, 6867, 16, 80, 124, 748], [913, 143, 79, 16, 6840, 1080, 809, 120], [156, 848, 13, 96, 1321, 6501, 171, 894], [66, 10, 831, 134, 656, 99, 7017, 1187], [12, 77, 142, 882, 122, 631, 1248, 6886]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.9090849084908 	 angle (deg): 0.0 	 threshold ge: 994.2850396137835
Qubit  (2)
ge fidelity (%): 82.20918691869184 	 angle (deg): 0.0 	 threshold ge: 899.297620004712
Qubit  (3)
ge fidelity (%): 71.21000000000002 	 angle (deg): 0.0 	 threshold ge: 764.0441324740764
thresholds=[994.2850396137835, 0, 899.297620004712, 764.0441324740764],
angles=[0.0, 0, 0.0, 0.0],
ge_avgs=[array([ 594.99475928,    0.        , 1412.27146726,    0.        ]), [0, 0, 0, 0], array([ 535.56567848,    0.        , 1318.567318  ,    0.        ]), array([ 484.11321742,    0.        , 1245.51369666,    0.        ])] ,
counts_calib=[[7058, 1207, 657, 128, 744, 116, 73, 17], [1174, 7049, 102, 721, 127, 742, 14, 71], [717, 119, 7029, 1190, 61, 15, 726, 143], [138, 809, 1111, 6994, 11, 83, 135, 719], [911, 140, 100, 14, 6866, 1081, 747, 141], [192, 913, 15, 96, 1260, 6692, 137, 695], [126, 21, 897, 169, 819, 152, 6606, 1210], [43, 166, 173, 947, 191, 885, 1126, 6469]]



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Saving S:\QRAM\qram_4QR2\data\data_241025\01099_QramProtocol3QTomo.h5


In [18]:
print('init_states =', init_states)
print(saved_files)

init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>']
['S:\\QRAM\\qram_4QR2\\data\\data_241025\\01020_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01021_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01022_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01023_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01024_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01025_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01026_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01027_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01028_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01029_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\01030_QramProtocol3QTomo.h5', 'S:\\QRAM\\

#### Retrieve saved files and put all the n_tomo_corrected into a more easily accessible file

In [21]:
n_tomo_corrected_dict = dict()
all_configs = dict()
for i_saved_file, saved_file in enumerate(tqdm(saved_files)):
    print(saved_file)
    data, attrs = prev_data(saved_file, None)
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    init_state = cfg.expt.init_state
    all_configs[init_state] = cfg
    play_pulses = cfg.expt.play_pulses
    print('init state Q0 (switch) Q1 (input):', init_state)
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=False)
    n_tomo_corrected = tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)
    
    if init_state not in n_tomo_corrected_dict.keys():
        n_tomo_corrected_dict.update({init_state:[]})
    
    n_tomo_corrected_dict[init_state].append(n_tomo_corrected)

init_states = np.copy(list(n_tomo_corrected_dict.keys()))
for init_state in init_states:
    all_n_tomo_corrected = np.array(n_tomo_corrected_dict[init_state])
    n_tomo_corrected_dict[init_state] = np.average(all_n_tomo_corrected, axis=0)

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_22060\3499015544.py: 3

  0%|          | 0/80 [00:00<?, ?it/s]

S:\QRAM\qram_4QR2\data\data_241025\01020_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\01020_QramProtocol3QTomo.h5


 C:\Users\slab\AppData\Local\Temp\ipykernel_22060\1887187011.py: 13

Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0>
S:\QRAM\qram_4QR2\data\data_241025\01021_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\01021_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|1>
S:\QRAM\qram_4QR2\data\data_241025\01022_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\01022_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+1>
S:\QRAM\qram_4QR2\data\data_241025\01023_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\01023_QramProtocol3QTomo.h5
Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
init state Q0 (switch) Q1 (input): |0>|0+i>
S:\QRAM\qram_4QR2\data\data_241025\01024_QramProtocol3QTomo.h5
S:\QRAM\qram_4QR2\data\data_241025\01024_QramProtocol3QTomo.h5
Extracte

In [22]:
# SAVING MEASURED n_tomo_corrected
n_tomo_corrected_dict.update({"saved_files":saved_files})
n_tomo_corrected_dict.update({"init_states":list(init_states)})
n_tomo_corrected_dict.update({"all_configs":all_configs})

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M")
qA, qB, qC = tomo_qubits
n_tomo_corrected_filepath = os.path.join(expt_path, f'{timestamp}_ntomocorrected_3Q_{qA}{qB}{qC}.npz')
np.savez(n_tomo_corrected_filepath, **n_tomo_corrected_dict, allow_pickle=True)
print(f'Saved n_tomo_corrected_dict for Q{qA} Q{qB} Q{qC} to file {n_tomo_corrected_filepath}')

Saved n_tomo_corrected_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412131029_ntomocorrected_3Q_023.npz


### Loop analysis

In [22]:
# saved_files = ['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00191_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00192_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00193_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00194_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00195_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00196_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00197_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00198_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00199_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00200_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00201_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00202_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00203_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00204_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00205_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00206_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00207_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00208_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00209_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00210_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00211_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00212_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00213_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00214_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00215_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00216_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00217_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00218_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00219_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00220_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00221_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00222_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00223_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00224_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00225_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00226_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00227_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00228_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00229_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00230_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00231_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00232_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00233_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00234_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00235_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00236_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00237_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00238_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00239_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00240_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00241_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00242_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00243_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00244_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00245_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00246_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00247_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00248_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00249_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00250_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00251_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00252_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00253_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00254_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00255_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00256_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00257_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00258_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00259_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00260_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00261_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00262_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00263_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00264_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00265_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00266_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00267_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00268_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00269_QramProtocol3QTomo.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00270_QramProtocol3QTomo.h5']

In [23]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
# rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202411051014_init_rhoMLE_ZZ_2Q_01.npz'
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412091502_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_241025\202412091502_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


In [24]:
print(rho_MLE_ZZ_2Q_filepath)

S:\QRAM\qram_4QR2\data\data_241025\202412091502_init_rhoMLE_ZZ_2Q_01.npz


In [25]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

Reopening 3q tomo data

In [124]:
# RETRIEVING MEASURED n_tomo_corrected
n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412090054_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412082023_ntomocorrected_3Q_023.npz"
# n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412090054_ntomocorrected_3Q_023.npz"
print(f"Using n_tomo_corrected_dict from file {n_tomo_corrected_filepath}")
n_tomo_corrected_dict = dict()

with np.load(n_tomo_corrected_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        n_tomo_corrected_dict.update({key:npzfile[key]})

init_states = n_tomo_corrected_dict["init_states"]

if 'saved_files' in init_states: init_states = init_states[:-1]

print(f'n_tomo_corrected_dict retrieved with init_states\n{init_states}')
all_configs = n_tomo_corrected_dict["all_configs"].item() #[0]
# print(all_configs.keys())


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Using n_tomo_corrected_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412090054_ntomocorrected_3Q_023.npz
n_tomo_corrected_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]


Looping MLE

In [125]:
use_init_2q_state = True

In [126]:
rhos_base = []
fidelities_base = []
purities_base = []

rhos_ZZ = []
fidelities_ZZ = []
purities_ZZ = []

In [127]:
for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected = n_tomo_corrected_dict[init_state]
    
    if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
    else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
    rho_id = get_qram_qSLR_state_from_rho(init_rho)
    
    # ------------------------------ #
    # Base MLE
    # ------------------------------ #
    
    rho_MLE_base = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=False,
    )
 
    fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    # print(f'Fidelity (base): {fid_base}')
    # print(f'Purity (base): {purity_base}')
    # print(f'Target purity: {purity_id}')
    
    # if saveplots: savetitle = saved_file[:-3]+f'_base{save_append}.svg'
    # else: savetitle = None
    # show_plot_rho_2d(rho_MLE_base, rho_id, title=f'Base MLE (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init {init_state}, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
    
    rhos_base.append(rho_MLE_base)
    fidelities_base.append(fid_base)
    purities_base.append(purity_base)
    
    rho_MLE = rho_MLE_base

    # ------------------------------ #
    # MLE with ZZ
    # ------------------------------ #

    rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats
    )

    fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    # print(f'Fidelity (with ZZ correction): {fid_ZZ}')
    # print(f'Purity (with ZZ correction): {purity_ZZ}')
    # print(f'Target purity: {purity_id}')

    # if saveplots: savetitle = saved_file[:-3]+f'_ZZcorrect{save_append}.svg'
    # else: savetitle = None
    # show_plot_rho_2d(rho_MLE_ZZ, rho_id, title=f'MLE with ZZ Correction (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init {init_state}, Play {play_pulses}', savetitle=savetitle, cmax=0.5)

    rhos_ZZ.append(rho_MLE_ZZ)
    fidelities_ZZ.append(fid_ZZ)
    purities_ZZ.append(purity_ZZ)

init state (Q0 (switch) Q1 (input)): |0>|0>
init state (Q0 (switch) Q1 (input)): |0>|1>
init state (Q0 (switch) Q1 (input)): |0>|0+1>
init state (Q0 (switch) Q1 (input)): |0>|0+i>
init state (Q0 (switch) Q1 (input)): |1>|0>
init state (Q0 (switch) Q1 (input)): |1>|1>
init state (Q0 (switch) Q1 (input)): |1>|0+1>
init state (Q0 (switch) Q1 (input)): |1>|0+i>
init state (Q0 (switch) Q1 (input)): |0+1>|0>
init state (Q0 (switch) Q1 (input)): |0+1>|1>
init state (Q0 (switch) Q1 (input)): |0+1>|0+1>
init state (Q0 (switch) Q1 (input)): |0+1>|0+i>
init state (Q0 (switch) Q1 (input)): |0+i>|0>
init state (Q0 (switch) Q1 (input)): |0+i>|1>
init state (Q0 (switch) Q1 (input)): |0+i>|0+1>
init state (Q0 (switch) Q1 (input)): |0+i>|0+i>


In [128]:
print('init_states=', init_states)
print(len(init_states))

init_states= ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
16


Run loop over virtual Z rotations

In [129]:
fids_grids_base_vs_meas = []
fids_grids_ZZ_vs_meas = []
fids_grids_base_vs_ideal = []
fids_grids_ZZ_vs_ideal = []

In [130]:
show_fids_grid = False

In [131]:
saveplots = False
# save_append = '_compareMeas'
save_append = ''

phis = [None, None, None]

In [132]:
phis[0] = np.linspace(0, 360, 20) # Q0
phis[1] = np.linspace(0, 360, 20) # Q2
phis[2] = np.linspace(0, 360, 20) # Q3

In [133]:
for use_init_2q_state in [True, False]:
    for use_ZZ_correction in [True, False]:
        print(f"STARTING use_init_2q_state: {use_init_2q_state}, use_ZZ_correction: {use_ZZ_correction}")
        fids_grids = np.zeros((len(init_states), len(phis[0]), len(phis[1]), len(phis[2])))
        fidelities_opt_rot = []
        all_best_phis = []
        for i_init_state, init_state in enumerate(tqdm(init_states)):
            # print('init state (Q0 (switch) Q1 (input)):', init_state)
            
            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
            rho_id = get_qram_qSLR_state_from_rho(init_rho)
            # ------------------------------ #
            # MLE with opt rot
            # ------------------------------ #
            if use_ZZ_correction:
                rho_MLE = rhos_ZZ[i_init_state]
            else:
                rho_MLE = rhos_base[i_init_state]
            
        
            # if init_state == '|0+1>|0+1>' or init_state == '|0+1>|1>': phis[0] = np.linspace(0, 360, 30)
                
            rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(
                qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis, progress=False, verbose=False)
            fids_grids[i_init_state] = fids_grid
            
            inner_sweep = phis[2]
            outer_sweep = phis[1]
            y_sweep = outer_sweep
            x_sweep = inner_sweep
        
            # vmax = np.max(fids_grid)
            vmax = 1.0
            # for i_phi0, phi0 in enumerate(phis[0]):
            if show_fids_grid:
                for i_phi0, phi0 in enumerate([phis[0][0]]):
                    print(fids_grid[i_phi0].shape)
                    print(f'phi Q0 {phi0}')
                    plt.figure()
                    plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})')
                    plt.xlabel(f'phis Q{tomo_qubits[2]}')
                    plt.ylabel(f'phis Q{tomo_qubits[1]}')
                    plt.pcolormesh(x_sweep, y_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
                    plt.colorbar()
                    plt.clim(vmin=0, vmax=vmax)
                    plt.show()
        
            z_phi123 = tomo_analysis.z_gate_nq(best_phis)
            rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
            fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2
            
            purity_MLE = np.real(np.trace(rho_MLE @ rho_MLE))
            purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
            purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
            # print(f'Fidelity (after rotation): {fid_rot}')
            # print(f'Purity (after rotation): {purity_MLE_rot}')
            # print(f'Target purity: {purity_id}')
            # print(f'Best rotation: {best_phis}')
            
            # if saveplots: savetitle=saved_file[:-3]+f'_ZZcorrect_optrot{save_append}.svg'
            # else: savetitle = None
            # show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init {init_state}, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
            
            fidelities_opt_rot.append(best_fid)
            all_best_phis.append(best_phis)
            
            # print('\n\n\n')
        if use_ZZ_correction and use_init_2q_state:
            fids_grids_ZZ_vs_meas = fids_grids
        elif use_ZZ_correction and not use_init_2q_state:
            fids_grids_ZZ_vs_ideal = fids_grids
        elif not use_ZZ_correction and use_init_2q_state:
            fids_grids_base_vs_meas = fids_grids
        elif not use_ZZ_correction and not use_init_2q_state:
            fids_grids_base_vs_ideal = fids_grids

STARTING use_init_2q_state: True, use_ZZ_correction: True


Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
 C:\Users\slab\AppData\Local\Temp\ipykernel_22060\665840663.py: 7

  0%|          | 0/16 [00:00<?, ?it/s]

STARTING use_init_2q_state: True, use_ZZ_correction: False


  0%|          | 0/16 [00:00<?, ?it/s]

STARTING use_init_2q_state: False, use_ZZ_correction: True


  0%|          | 0/16 [00:00<?, ?it/s]

STARTING use_init_2q_state: False, use_ZZ_correction: False


  0%|          | 0/16 [00:00<?, ?it/s]

#### Save fids grids

In [136]:
fids_grids_dict = dict(
    phis=phis,
    fids_grids_base_vs_meas=fids_grids_base_vs_meas,
    fids_grids_ZZ_vs_meas=fids_grids_ZZ_vs_meas,
    fids_grids_base_vs_ideal=fids_grids_base_vs_ideal,
    fids_grids_ZZ_vs_ideal=fids_grids_ZZ_vs_ideal, 
)
fids_grids_dict.update(n_tomo_corrected_dict)
if "allow_pickle" in fids_grids_dict: fids_grids_dict.pop("allow_pickle")

In [137]:
# SAVING FIDS GRIDS
# n_tomo_corrected_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412131029_ntomocorrected_3Q_023.npz'
fids_grids_filepath = n_tomo_corrected_filepath[:-4] + '_fids_grids.npz'
np.savez(fids_grids_filepath, **fids_grids_dict)
print(f'Saved fids_grids_dict for Q{qA} Q{qB} Q{qC} to file {fids_grids_filepath} with keys\n{fids_grids_dict.keys()}')

Saved fids_grids_dict for Q0 Q2 Q3 to file S:\QRAM\qram_4QR2\data\data_241025\202412090054_ntomocorrected_3Q_023_fids_grids.npz with keys
dict_keys(['phis', 'fids_grids_base_vs_meas', 'fids_grids_ZZ_vs_meas', 'fids_grids_base_vs_ideal', 'fids_grids_ZZ_vs_ideal', '|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>', 'saved_files', 'init_states', 'all_configs'])


Reopening fids grids and 3q tomo data

In [138]:
# RETRIEVING MEASURED n_tomo_corrected and fids grids
# fids_grids_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412131029_ntomocorrected_3Q_023_fids_grids.npz"
print(f"Using fids_grids_dict from file {fids_grids_filepath}")
fids_grids_dict = dict()

with np.load(fids_grids_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        fids_grids_dict.update({key:npzfile[key]})

init_states = fids_grids_dict["init_states"]

if 'saved_files' in init_states: init_states = init_states[:-1]

print(f'fids_grids_dict retrieved with init_states\n{init_states}')
all_configs = fids_grids_dict["all_configs"].item() #[0]
# print(all_configs.keys())


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412090054_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]


Select the fids grids you want to use to display/calculate best average fidelities

In [139]:
use_init_2q_state = False
use_ZZ_correction = True

In [140]:
if use_ZZ_correction and use_init_2q_state:
    fids_grids = fids_grids_dict["fids_grids_ZZ_vs_meas"]
elif use_ZZ_correction and not use_init_2q_state:
    fids_grids = fids_grids_dict["fids_grids_ZZ_vs_ideal"]
elif not use_ZZ_correction and use_init_2q_state:
    fids_grids = fids_grids_dict["fids_grids_base_vs_meas"]
elif not use_ZZ_correction and not use_init_2q_state:
    fids_grids = fids_grids_dict["fids_grids_base_vs_ideal"]

In [141]:
print(np.array(fids_grids).shape)
print(len(init_states))

(16, 20, 20, 20)
16


Go through the fids grids and select the best overall phi

In [142]:
check_init_states = init_states
# check_init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>']
# check_init_states = ['|0+1>|1>', '|1>|0+1>', '|0>|0+1>', '|0+1>|0+1>']
# check_init_states = ['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+i>']

In [144]:
# Pick best overall phi

# check_init_states = ['|1>|0+1>', '|0>|0+1>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0>']
# check_init_states = ['|1>|0+1>', '|0>|0+1>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0>']
# check_init_states = ['|1>|0+1>', '|0>|0+1>', '|0+1>|1>', '|0+1>|0>', '|0+1>|0+1>']
# check_init_states = ['|1>|0+1>', '|0>|0+1>', '|0+1>|1>']
# check_init_states = ['|0+1>|0+1>']
    

fids_grids_reshaped = []
for i_state in range(len(init_states)):
    grid_shape = fids_grids[i_state].shape
    # print(grid_shape)
    fids_grid = np.copy(fids_grids[i_state]).tolist()
    # print(init_states[i_state], grid_shape)
    if grid_shape[0] == 1:
        for i in range(grid_shape[1]-1):
            fids_grid.append(fids_grid[0])
    # print(np.array(fids_grid).shape)
    fids_grids_reshaped.append(fids_grid)
    
fids_grids_reshaped = np.array(fids_grids_reshaped)
print(fids_grids_reshaped.shape)


best_avg_fid = 0
best_fids = []
best_phis = [0, 0, 0]
best_phi_indices = [0, 0, 0]
individual_best_fids = np.zeros((len(init_states)))
individual_best_phis = np.zeros((len(init_states), 3))
for iphi0, phi0 in enumerate(phis[0]):
    for iphi1, phi1 in enumerate(phis[1]):
        for iphi2, phi2 in enumerate(phis[2]):
            avg_fid = 0
            for i_state, init_state in enumerate(init_states):
                if init_state not in check_init_states: continue
                fid = fids_grids_reshaped[i_state][iphi0, iphi1, iphi2]
                avg_fid += fid
                if fid > individual_best_fids[i_state]:
                    individual_best_fids[i_state] = fid
                    individual_best_phis[i_state] = [phi0, phi1, phi2]
            avg_fid /= len(check_init_states)
            if avg_fid > best_avg_fid:
                best_phis = [phi0, phi1, phi2]
                best_phi_indices = [iphi0, iphi1, iphi2]
                best_avg_fid = avg_fid
 
vmax = 1.0
for i_state in range(len(init_states)):
    # print(init_states[i_state])
    # print(f'best phi Q0 {best_phis[0]}')
    plt.figure()
    # plt.title(f'Fidelities {init_states[i_state]} ($\phi_0: ${best_phis[0]:0.4})')
    plt.xlabel(f'phis Q{tomo_qubits[2]}')
    plt.ylabel(f'phis Q{tomo_qubits[1]}')
    plt.pcolormesh(x_sweep, y_sweep, fids_grids_reshaped[i_state, best_phi_indices[0], :, :], cmap='viridis', shading='auto')
    plt.colorbar()
    plt.clim(vmin=0, vmax=vmax)
    plt.show()


for i_state, init_state in enumerate(init_states):
    if init_state not in check_init_states: continue
    best_fids.append(fids_grids_reshaped[i_state][best_phi_indices[0], best_phi_indices[1], best_phi_indices[2]])

print('check states', check_init_states)
print('Best fids with individual rotations', individual_best_fids.tolist())
print('Avg of best fids with individual rotations', np.average(individual_best_fids))
print('Phis for best fids with individual rotations', individual_best_phis.tolist())
print('Best avg fid with overall optimal phis', best_avg_fid)
print('Fids for check states using overall optimal phis', best_fids)
print('Overall optimal phis', best_phis)

(16, 20, 20, 20)
check states ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
Best fids with individual rotations [0.9705727957765565, 0.8042358280084659, 0.9010165030318109, 0.9091766689130935, 0.9167999419589756, 0.6530650560121191, 0.8488716454060631, 0.8623007188853337, 0.9180946185316663, 0.7503055713497813, 0.8372031378049452, 0.8384224760868983, 0.9314052864962525, 0.7391120539779328, 0.8492586389426062, 0.8587951712008605]
Avg of best fids with individual rotations 0.8492897570239601
Phis for best fids with individual rotations [[18.94736842105263, 189.4736842105263, 341.05263157894734], [265.2631578947368, 151.57894736842104, 113.68421052631578], [132.6315789473684, 113.68421052631578, 0.0], [208.42105263157893, 113.68421052631578, 189.4736842105263], [151.57894736842104, 18.94736842105263, 341.05263157894734], [170.52631578947367, 56.842105263157

Plot all the tomos for the best phis

In [713]:
# best_phis = [360.0, 360.0, 235.86206896551724]
# best_phis = [0]*3

In [696]:
saveplots = False

In [697]:
z_phi123 = tomo_analysis.z_gate_nq(best_phis)
best_fids = []

print()
for i_state, init_state in enumerate(init_states):
    # if init_state not in check_init_states: continue
    print('init_state', init_state)
    if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
    else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
    rho_id = get_qram_qSLR_state_from_rho(init_rho)
    
    if use_ZZ_correction: rho_MLE_rot = (z_phi123*qt.Qobj(rhos_ZZ[i_state], dims=rho_id.dims)*z_phi123.dag()).unit()
    else: rho_MLE_rot = (z_phi123*qt.Qobj(rhos_base[i_state], dims=rho_id.dims)*z_phi123.dag()).unit()
    fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2
    best_fids.append(fid_rot)
    
    purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    print(f'Fidelity (after rotation): {fid_rot}')
    print(f'Purity (after rotation): {purity_MLE_rot}')
    print(f'Target purity: {purity_id}')
    
    if saveplots: savetitle=saved_file[:-3]+f'{"_ZZcorrect" if use_ZZ_correction else ""}_optrot{save_append}.svg'
    else: savetitle = None
    
    init_state_title = ''
    for c in init_state:
        if c != '>': init_state_title += c
        else: init_state_title += '\\rangle'
    tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE {"with ZZ" if use_ZZ_correction else "Base"}, Phase Corrected (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})\nInit ${init_state_title}$, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
    print()
print('init_states =', init_states)
print('best fids')
print(best_fids)
print('average best fid', np.average(best_fids))
print('used best_phis', best_phis)


init_state |0>|0>
constructing final state on |switch, out1, out2>
Fidelity (after rotation): 0.9845468605859264
Purity (after rotation): 0.9480939929882424
Target purity: 0.9801405324149476

init_state |0>|1>
constructing final state on |switch, out1, out2>
Fidelity (after rotation): 0.9503034934343056
Purity (after rotation): 0.8811709187771561
Target purity: 0.9881482076664617

init_state |0>|0+1>
constructing final state on |switch, out1, out2>
Fidelity (after rotation): 0.8541556176068137
Purity (after rotation): 0.8575568176942183
Target purity: 0.9897439727843643

init_state |0>|0+i>
constructing final state on |switch, out1, out2>
Fidelity (after rotation): 0.8477877103107541
Purity (after rotation): 0.8804967344358815
Target purity: 0.9515414421858257

init_state |1>|0>
constructing final state on |switch, out1, out2>
Fidelity (after rotation): 0.9585268720224344
Purity (after rotation): 0.9880305257738403
Target purity: 0.9250320745837146

init_state |1>|1>
constructing fina

## Run analysis

### Reopen old data

In [20]:
apply_ps = False
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [125]:
saved_files = []
filename = 'S:\QRAM\qram_4QR2\data\data_241025\\00272_QramProtocol3QTomo.h5'
# data, attrs = prev_data(expt_path, '00009_QramProtocol3QTomo.h5')
saved_files.append(filename)
data, attrs = prev_data(filename, None)
pulse_dict = attrs['pulse_dict']
print('calib_order', attrs['calib_order'])
print('meas_order', attrs['meas_order'])

print(data.keys())
# # data = None
# # style.use('default')
# # qramExpt.analyze(data=data)
# # qramExpt.display(data=data, err=False, saveplot=True)
# # print(data)

cfg = AttrDict(attrs['config'])
init_state = cfg.expt.init_state
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits
print('init_state', init_state)
print('play_pulses', play_pulses)
print('tomo_qubits', tomo_qubits)

np.set_printoptions(suppress=True)
n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)
print('counts_calib')
n_calib = n_calib.astype(np.float64)
for r in range(n_calib.shape[0]):
    n_calib[r,:]/=np.sum(n_calib[r,:])
print(n_calib)
print(attrs['calib_order'])

print(f'thresholds={data["thresholds"].tolist()},')
print(f'angles={data["angles"].tolist()},')
print(f'ge_avgs={data["ge_avgs"].tolist()}',',')
print(f'counts_calib={data["counts_calib"].tolist()}')

S:\QRAM\qram_4QR2\data\data_241025\00272_QramProtocol3QTomo.h5


 C:\Users\slab\AppData\Local\Temp\ipykernel_10520\809401443.py: 13

Extracted data and attrs ['calib_order', 'config', 'meas_order', 'pulse_dict']
calib_order ['ggg', 'gge', 'geg', 'gee', 'egg', 'ege', 'eeg', 'eee']
meas_order ['ZZZ', 'ZZX', 'ZZY', 'ZXZ', 'ZXX', 'ZXY', 'ZYZ', 'ZYX', 'ZYY', 'XZZ', 'XZX', 'XZY', 'XXZ', 'XXX', 'XXY', 'XYZ', 'XYX', 'XYY', 'YZZ', 'YZX', 'YZY', 'YXZ', 'YXX', 'YXY', 'YYZ', 'YYX', 'YYY']
dict_keys(['angles', 'calib_ishots_raw', 'calib_qshots_raw', 'counts_calib', 'counts_raw', 'counts_tomo', 'epop', 'fpop', 'ge_avgs', 'gpop', 'ishots_raw', 'qshots_raw', 'thresholds', 'xpts'])
init_state |0>|1>
play_pulses [2, 1, 2, 1, 2, 1, 4, 3]
tomo_qubits [0, 2, 3]
counts_calib
[[0.7016 0.1161 0.0819 0.0124 0.063  0.0137 0.0091 0.0022]
 [0.1121 0.7027 0.0112 0.082  0.011  0.0682 0.0017 0.0111]
 [0.0973 0.016  0.6827 0.1121 0.0104 0.0017 0.0664 0.0134]
 [0.0204 0.1035 0.1189 0.6655 0.0014 0.0117 0.0131 0.0655]
 [0.0668 0.0141 0.007  0.0015 0.6385 0.1539 0.0918 0.0264]
 [0.0139 0.0545 0.0019 0.007  0.1474 0.6423 0.0267 0.1063]
 [0.0117 0.002 

In [ ]:
# # counts_calib = data['counts_calib']
# counts_calib=[array([3189,  426, 3176,  519, 1176,  198, 1110,  206]), array([ 580, 3047,  582, 3015,  207, 1148,  221, 1200]), array([1963,  289, 4456,  583,  721,   97, 1661,  230]), array([ 390, 1850,  812, 4372,  130,  683,  269, 1494]), array([1336,  209, 1293,  232, 2954,  473, 2981,  522]), array([ 256, 1341,  226, 1320,  512, 2924,  557, 2864]), array([ 809,  118, 1892,  295, 1810,  262, 4215,  599]), array([ 167,  796,  352, 1830,  325, 1776,  781, 3973])]


# # introduce gaussian noise
# np.random.seed(0)
# noise = 2.0
# n = np.copy([counts_calib[4]])
# print(n, sum(n))
# n0 = correct_readout_err(n, n_conf=counts_calib)
# print('correct readout err', n0)
# print('fix neg counts', fix_neg_counts(n0))

# if noise is not None:
#     for n_psi in n:
#         n_meas = sum(n_psi)
#         new_nlast = -1
#         while new_nlast < 0:
#                 new_n_uptolast = np.random.normal(loc=n_psi[:-1], scale=noise*np.sqrt(n_psi[:-1]))
#                 # new_n_uptolast *= 1 - np.isnan(new_n_uptolast) # 0 out the nans
#                 # print('here', new_n_uptolast)
#                 # preserve original total count per measurement
#                 new_nlast = n_meas - sum(new_n_uptolast)
#                 n_psi[:-1] = np.round(new_n_uptolast)
#                 n_psi[-1] = np.round(new_nlast)

# print(1)
# print(n, sum(n))
# print(type(n))
# print(np.shape(n))
# print(n[0].dtype)
# n = np.array(n, dtype=float)
# n1 = correct_readout_err(n, n_conf=counts_calib)
# print('correct readout err', n1)
# print('fix neg counts', fix_neg_counts(n1))


# # print('n1-n2')
# # print(n1-n2)

# print(3)
# n = [[ 287, 1291,  675, 2954,  256, 1250,  554, 2733]]
# print(n)
# n = correct_readout_err(n, n_conf=counts_calib)
# print(sum(n))
# print(fix_neg_counts(n))

# print(4)
# correct_readout_err(counts_calib, counts_calib)

### Set ideal rho

Get ideal psi for fidelity calculation. Change this depending on what you want for ideal final state.

#### Get ideal final qram state from an arbitrary 2Q input density matrix

In [189]:
print('init state Q0 (switch) Q1 (input):', init_state)

init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
# if init_state == '|0+1>|0+1>':
#     init_rho = np.array([[(0.23+0j), (0.057+0.233j), (0.18-0.099j), (-0.078+0.08j)], [(0.057-0.233j), (0.287+0j), (-0.016-0.171j), (-0.013+0.114j)], [(0.18+0.099j), (-0.016+0.171j), (0.266+0j), (-0.166+0.121j)], [(-0.078-0.08j), (-0.013-0.114j), (-0.166-0.121j), (0.217+0j)]])
# init_rho = qt.ket2dm((name_to_state_2q('|1>|0>') + name_to_state_2q('|1>|1>') + name_to_state_2q('|0>|0>') + 1j*name_to_state_2q('|0>|1>')).unit())

rho_id = get_qram_qSLR_state_from_rho(init_rho)
# rho_id


# # rho_MLE_ZZ_2Q_filepath = 'S:\Connie\experiments\qramLL_4QR2\data\data_240218\\202402281619_init_rhoMLE_ZZ_2Q_01.npz'
# rho_MLE_ZZ_2Q_filepath = 'S:\Connie\experiments\qramLL_4QR2\data\data_240218\\202402291250_init_rhoMLE_ZZ_2Q_01.npz'

# print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

# rho_MLE_ZZ_dict = dict()
# with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
#     for key in npzfile.keys():
#         rho_MLE_ZZ_dict.update({key:npzfile[key]})
# print(f'rho_MLE_ZZ retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

# init_rho = rho_MLE_ZZ_dict[init_state]
# rho_id = get_qram_qSLR_state_from_rho(init_rho)

init state Q0 (switch) Q1 (input): |0+i>|1>
constructing final state on |switch, out1, out2>


In [88]:
# psi_id = qt.tensor(psiZ[1]+psiZ[0], psiZ[0], psiZ[1]+psiZ[0]).unit()
# psi_id = qt.tensor(psiZ[1], psiZ[0], psiZ[1]).unit()
# rho_id = qt.ket2dm(psi_id)
# rho_id = qt.ket2dm(psis['010'] + psis['011']).unit()
rho_id = qt.ket2dm(psis['010']).unit()
# print(rho_id.dims)

### Generate test data

In [ ]:
# rho_id = qt.ket2dm(psi10 + psi11).unit()

# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, noise=None)
n_tomo_sim = generate_counts(rho_id=rho_id, n_tot=100000, evol_mats=evol_mats, noise=None)
# n_tomo = generate_counts(rho_id=rho_MLE_ZZ, n_tot=100000, evol_mats=evol_mats, noise=None)
# n_tomo = n_tomo_sim
print(n_tomo_sim)

print(meas_order)
print(calib_order)
print('difference')
print(np.round((n_tomo_sim - n_tomo)/100000, 2))

In [ ]:
# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, noise=None)
# n_tomo = generate_counts(rho_id=rho_id, n_tot=8000, evol_mats=evol_mats, noise=None)
# n_calib = qt.qeye(4).full()
# print(fix_neg_counts(n_tomo))

### Run MLE without ZZ correction

In [363]:
# methods = 'Nelder-Mead' 'Powell' 'CG' 'BFGS' 'Newton-CG' 'L-BFGS-B' 'TNC' 'COBYLA' 'SLSQP' 'dogleg' 'trust-ncg'
# rho_MLE_base = run_MLE(n=n_tomo, rho_id=rho_id, ZZ_correction=False, method='L-BFGS-B')
rho_MLE_base = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=False,
)

print(rho_MLE_base)

[[ 0.31285536-0.j         -0.02458826+0.00383627j -0.21223837-0.15402228j
   0.00036948-0.01137754j  0.28455813+0.03151448j -0.11108474-0.14541526j
   0.04233061+0.00954858j -0.00365875-0.00886387j]
 [-0.02458826-0.00383627j  0.01948968-0.j          0.01428835+0.00007306j
   0.00248009-0.00156461j -0.01976387+0.01244095j -0.00152893-0.01149484j
   0.00026836-0.00476889j -0.00094367+0.00272903j]
 [-0.21223837+0.15402228j  0.01428835-0.00007306j  0.23205322+0.j
   0.00732569+0.01018456j -0.22400449+0.12003321j  0.16748949+0.03757418j
  -0.03059293+0.01736975j  0.005217  +0.0032166j ]
 [ 0.00036948+0.01137754j  0.00248009+0.00156461j  0.00732569-0.01018456j
   0.00115884-0.j         -0.0030551 +0.01348031j  0.00727887-0.00907311j
   0.00071934+0.00150889j -0.00013035-0.00000022j]
 [ 0.28455813-0.03151448j -0.01976387-0.01244095j -0.22400449-0.12003321j
  -0.0030551 -0.01348031j  0.28162467+0.j         -0.14228654-0.11523314j
   0.0362249 +0.0003221j  -0.00227329-0.00626226j]
 [-0.11108474

In [364]:
fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
print(f'Fidelity (no ZZ correction): {fid_base}')
print(f'Purity (no ZZ correction): {purity_base}')
print(np.around(rho_MLE_base, decimals=3))

init_state_title = ''
for c in init_state:
    if c != '>': init_state_title += c
    else: init_state_title += '\\rangle'
tomo_analysis.show_plot_rho_2d(rho_MLE_base, rho_id, title=f'Base MLE (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})\nInit ${init_state_title}$, Play {play_pulses}', savetitle=None, cmax=0.5)

print('Saved data:')
print(*saved_files)

Fidelity (no ZZ correction): 0.04372193906305965
Purity (no ZZ correction): 0.8911692474860778
[[ 0.313-0.j    -0.025+0.004j -0.212-0.154j  0.   -0.011j  0.285+0.032j
  -0.111-0.145j  0.042+0.01j  -0.004-0.009j]
 [-0.025-0.004j  0.019-0.j     0.014+0.j     0.002-0.002j -0.02 +0.012j
  -0.002-0.011j  0.   -0.005j -0.001+0.003j]
 [-0.212+0.154j  0.014-0.j     0.232+0.j     0.007+0.01j  -0.224+0.12j
   0.167+0.038j -0.031+0.017j  0.005+0.003j]
 [ 0.   +0.011j  0.002+0.002j  0.007-0.01j   0.001-0.j    -0.003+0.013j
   0.007-0.009j  0.001+0.002j -0.   -0.j   ]
 [ 0.285-0.032j -0.02 -0.012j -0.224-0.12j  -0.003-0.013j  0.282+0.j
  -0.142-0.115j  0.036+0.j    -0.002-0.006j]
 [-0.111+0.145j -0.002+0.011j  0.167-0.038j  0.007+0.009j -0.142+0.115j
   0.145-0.j    -0.016+0.023j  0.003-0.001j]
 [ 0.042-0.01j   0.   +0.005j -0.031-0.017j  0.001-0.002j  0.036-0.j
  -0.016-0.023j  0.007-0.j    -0.001-0.001j]
 [-0.004+0.009j -0.001-0.003j  0.005-0.003j -0.   +0.j    -0.002+0.006j
   0.003+0.001j -0.00

In [129]:
print()

### Run MLE with ZZ

#### Get evol mats for ZZ

In [21]:
# Recover old config
yaml_cfg = AttrDict(attrs['config'])

In [ ]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [29]:
tomo_qubits = [0, 2, 3]

In [30]:
# evol_mats_path = os.path.join(os.getcwd(), 'evol_mats')
evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
print('Will save evol mats to path', evol_mats_path)

qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'
# evol_mats_filename = f'evol_mats_{qA}{qB}{qC}_old.npz'
print('Will save to filename', evol_mats_filename)

Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats
Will save to filename evol_mats_023.npz


In [37]:
evol_mats = tomo_analysis.get_evol_mats_from_yaml(
    tomo_qubits=tomo_qubits,
    yaml_cfg=yaml_cfg,
    pulse_dict=pulse_dict,
    cutoffs=[4]*(len(tomo_qubits)),
    soccfg=soc,
    debug=False,
    evol_mats_path=evol_mats_path,
    evol_mats_filename=evol_mats_filename,
)

TOMO QUBITS [0, 2, 3]
HAMILTONIAN QUBITS [0 2 3]
qubit freqs [4111.09786656 4762.90947354 4381.24812102]
alphas [-226.47095044 -189.7408626  -171.32303476]
ZZs (MHz) [[ 0.         -0.13971326 -0.55145871]
 [-0.14796941  0.         -0.53966012]
 [-0.54951646 -0.53028391  0.        ]]
Will save to filename evol_mats_023.npz


  0%|          | 0/27 [00:00<?, ?it/s]

Saved evol mats to file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


#### Reload evol mats

In [31]:
qA, qB, qC = tomo_qubits
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})
# print(evol_mats)

Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


#### (Actually) run MLE with ZZ correction

In [192]:
rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=True,
    evol_mats=evol_mats
)

In [25]:
# GET IDEAL RHO FROM DICTIONARY
print('init state to get ideal rho', init_state)
init_rho = rho_MLE_ZZ_dict[init_state]
rho_id = get_qram_qSLR_state_from_rho(init_rho)

init state to get ideal rho |0+1>|1>
constructing final state on |switch, out1, out2>


In [193]:
print(np.around(rho_MLE_ZZ, decimals=3))
fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (with ZZ correction): {fid_ZZ}')
print(f'Purity (with ZZ correction): {purity_ZZ}')
print(f'Target purity: {purity_id}')
# savetitle = '1_0+1_ideal.png'
# savetitle = 'test.png'
savetitle=None

# if savetitle is None: plt.style.use('default')
# show_plot_rho_2d(rho_id, rho_id, title='MLE with ZZ Correction', savetitle=savetitle) #, cmax=1.0)
print('init state', init_state)
init_state_title = ''
for c in init_state:
    if c != '>': init_state_title += c
    else: init_state_title += '\\rangle'
tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ, rho_id, title=f'MLE with ZZ Correction (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init ${init_state_title}$, Play {play_pulses}', savetitle=savetitle, cmax=0.5)

if save_data:
    print('Saved data:')
    print(saved_files[-1])

[[ 0.002-0.j    -0.   +0.004j -0.014+0.004j -0.   +0.j    -0.003+0.003j
   0.013-0.013j -0.004-0.001j -0.   +0.j   ]
 [-0.   -0.004j  0.014-0.j    -0.   +0.032j  0.   +0.001j  0.011+0.008j
  -0.026-0.03j  -0.005+0.01j   0.002+0.001j]
 [-0.014-0.004j -0.   -0.032j  0.485-0.j    -0.004+0.001j  0.022-0.001j
  -0.445+0.115j  0.077+0.045j  0.001+0.001j]
 [-0.   -0.j     0.   -0.001j -0.004-0.001j  0.   -0.j     0.001-0.001j
   0.003+0.002j -0.   -0.j     0.   -0.j   ]
 [-0.003-0.003j  0.011-0.008j  0.022+0.001j  0.001+0.001j  0.016-0.j
  -0.038+0.014j  0.004+0.008j  0.002-0.001j]
 [ 0.013+0.013j -0.026+0.03j  -0.445-0.115j  0.003-0.002j -0.038-0.014j
   0.465-0.j    -0.057-0.067j -0.003-0.002j]
 [-0.004+0.001j -0.005-0.01j   0.077-0.045j -0.   +0.j     0.004-0.008j
  -0.057+0.067j  0.019-0.j     0.   -0.001j]
 [-0.   -0.j     0.002-0.001j  0.001-0.001j  0.   +0.j     0.002+0.001j
  -0.003+0.002j  0.   +0.001j  0.   -0.j   ]]
Fidelity (with ZZ correction): 0.3599306274893872
Purity (with ZZ 

### Optimize over virtual Z gates on ideal rho 

Optimize over virtual Z gates on ideal rho 

In [741]:
# rho_MLE = rho_MLE_base
rho_MLE = rho_MLE_ZZ

In [742]:
phis = [None, None, None]
phis[0] = np.linspace(0, 360, 30)
phis[1] = np.linspace(0, 360, 30)
phis[2] = np.linspace(0, 360, 30)
# phis[0] = np.linspace(0, 360, 1)
# phis[1] = np.linspace(0, 360, 100)
# phis[2] = np.linspace(0, 360, 100)

rho_MLE_rot, best_phis, best_fid, fids_grid = opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis)
print('Rotation:', best_phis)

NameError: name 'opt_virtualZ_MLE' is not defined

In [ ]:
inner_sweep = phis[2]
outer_sweep = phis[1]
y_sweep = outer_sweep
x_sweep = inner_sweep

# vmax = np.max(fids_grid)
vmax = 1.0

# best_phis =  [350.7692307692307, 304.6153846153846, 138.46153846153845]
# best_phis =  [115, 350, 203]
# best_phis = [350.7692307692307, 18.46153846153846, 258.46153846153845]

saveplot = True
plot_filename = filename[:-3]+'.svg'
if saveplot: plt.style.use('dark_background')
best_phi_inds = [np.argmin(np.abs(phis[i]-best_phis[i])) for i in range(3)]

for i_phi0, phi0 in enumerate(phis[0]):
    if i_phi0 != best_phi_inds[0]: continue 
    print(f'phi Q0 {phi0}')
    plt.figure()
    plt.title(f'Fidelities ($\phi_0: ${phi0:0.4})', fontsize=18)
    plt.xlabel(f'$\phi_{tomo_qubits[2]}$', fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel(f'$\phi_{tomo_qubits[1]}$', fontsize=18)
    plt.yticks(fontsize=14)
    plt.pcolormesh(y_sweep, x_sweep, fids_grid[i_phi0], cmap='viridis', shading='auto')
    
    plt.plot([best_phis[2]], [best_phis[1]], 'r', marker='*', markersize=15)
    print(f'fid at phi2={best_phis[1]}, phi3={best_phis[2]}:', fids_grid[i_phi0][best_phi_inds[1]][best_phi_inds[2]])
    
    # old_best_phis = [0.0, 94.54545454545455, 349.09090909090907]
    # old_best_phis = [0.0, 196.36363636363635, 341.8181818181818]
    # plt.plot([old_best_phis[2]], [old_best_phis[1]], 'r', marker='o')
    
    divider = make_axes_locatable(plt.gca())
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(cax=cax, ticks=[0, 0.5, vmax])
    # cbar.ax.set_title(f'Fidelity')
    cbar.ax.tick_params(labelsize=12)
    plt.clim(vmin=0, vmax=vmax)
    
    if saveplot:
        plt.savefig(plot_filename, format='svg', bbox_inches='tight', transparent=True)
        print('Saved', plot_filename)
        
    plt.show()
    
# print(f'Difference between phi0[0] and phi0[1]')
# plt.figure()
# plt.title(f'Difference in fidelities ($\phi_0: $0 vs 30)')
# plt.xlabel(f'phis Q{tomo_qubits[2]}')
# plt.ylabel(f'phis Q{tomo_qubits[1]}')
# plt.pcolormesh(y_sweep, x_sweep, fids_grid[0]-fids_grid[1], cmap='viridis', shading='auto')

# # plt.plot([best_phis[2]], [best_phis[1]], 'r', marker='o')

# # old_best_phis = [0.0, 94.54545454545455, 349.09090909090907]
# # old_best_phis = [0.0, 196.36363636363635, 341.8181818181818]
# # plt.plot([old_best_phis[2]], [old_best_phis[1]], 'r', marker='o')

# plt.colorbar()
# # plt.clim(vmin=0, vmax=1)
# plt.show()

In [ ]:
best_phis = [360.0, 360.0, 235.86206896551724]
# best_phis = old_best_phis

# rho_MLE = rho_MLE_ZZ

z_phi123 = tomo_analysis.z_gate_nq(best_phis)
rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2

purity_MLE = np.real(np.trace(rho_MLE @ rho_MLE))
purity_MLE_rot = np.real(np.trace(rho_MLE_rot.full() @ rho_MLE_rot.full()))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (after rotation): {fid_rot}')
print(f'Purity (after rotation): {purity_MLE_rot}')
print(f'Target purity: {purity_id}')
print(f'Best rotation: {best_phis}')

# print(np.around(rho_MLE_rot, decimals=3))
savetitle=None
# savetitle=saved_file[:-3]+f'_ZZcorrect_optrot.svg'
tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=savetitle, cmax=0.5)
print('Saved data')
print(saved_files[-1])

# Loop over 2Q sweep parameters

In [ ]:
# gain_sweep = np.linspace(2000, 25000, 11)
# opt_lens = 1e3*np.array([2.4201237152470587, 1.1987019004033739, 0.7868408797215013, 0.5851160587684981, 0.4716335769416996, 0.3999234927074055, 0.34781134410506404, 0.3089041362621608, 0.2818412962269934, 0.2636215545192442, 0.2512183319921639])
# opt_freqs = [5891.5, 5891.325, 5890.9, 5890.35, 5889.675, 5888.875, 5887.95, 5887.15, 5886.1, 5885.55, 5884.5]


gain_sweep = np.linspace(18000, 32000, 11)
opt_freqs = [5887.35, 5886.616666666667, 5885.883333333333, 5885.616666666667, 5885.116666666667, 5884.383333333333, 5884.116666666667, 5883.616666666667, 5882.883333333333, 5882.383333333333, 5882.583333333333]
opt_lens = [0.30514258121790533, 0.28952631508344284, 0.27703044436306967, 0.2646698856191053, 0.25368901625658447, 0.24735611563570364, 0.23874769989670325, 0.23044507614293083, 0.22816368954972915, 0.22170985331767395, 0.20962837376879714]

save_data = True
sweep2Q_q = 2
qubit = 2
play_pulses=[1,3]

In [ ]:
saved_files = []

init_state = '|0+1>|1>'

opt_phis = [None]

thresholds = None
angles = None
ge_avgs = None
counts_calib = None


for i_gain, gain in enumerate(gain_sweep):
    
    tomo = meas.QramProtocol3QTomoExperiment(
        soccfg=soc,
        path=expt_path,
        config_file=config_path,
    )
    
    span = np.inf
    npts = 1
    if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))
    
    tomo.cfg.expt = dict(
        start=span,
        step=0,
        expts=1, # number of different time experiments
        reps=30000, # number of reps per tomo/time step,
        singleshot_reps=30000, # reps per state for singleshot calibration
        post_process='scale', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
        tomo_3q=True,
        tomo_qubits=tomo_qubits,
        init_state=init_state,
        play_pulses=play_pulses,
        thresholds=thresholds,
        angles=angles,
        ge_avgs=ge_avgs,
        counts_calib=counts_calib,
    )
    tomo.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = int(gain)
    tomo.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] = opt_lens[i_gain]
    tomo.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = opt_freqs[i_gain]
    print('gain', tomo.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q],
          'len', tomo.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q],
          'freq', tomo.cfg.device.qubit.f_EgGf_Q[sweep2Q_q])
    try:
        tomo.go(analyze=False, display=False, progress=False, save=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
    
    if save_data: saved_files.append(tomo.save_data())
    data = tomo.data
    if i_gain == 0:
        thresholds = data['thresholds']
        angles = data['angles']
        ge_avgs = data['ge_avgs']
        counts_calib = data['counts_calib']
    
    print()

In [ ]:
print(saved_files)

In [ ]:
print(saved_files)

### Calculate fidelities with ZZ correction and phase optimization

In [ ]:
apply_ps = True
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]

In [ ]:
all_n_tomo = []
all_n_calib = []
all_fids = []
all_fids_rot = []

rho_id = qt.ket2dm(psis['001'] + psis['110']).unit()

for saved_file in saved_files:
    saved_file = saved_file.split('\\')
    # print(saved_file)
    data, attrs = prev_data(expt_path, saved_file[-1])
    pulse_dict = attrs['pulse_dict']
    cfg = AttrDict(attrs['config'])
    
    n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)

    all_n_tomo.append(n_tomo_raw)
    all_n_calib.append(n_calib)

    rho_MLE = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_raw,
        n_calib=n_calib,
        correct_readout=True,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats,
    )
    
    # show_plot_rho_2d(rho_MLE_ZZ, rho_id, title=f'MLE with ZZ Correction (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=savetitle, cmax=0.5)
    fid_base = qt.fidelity(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"

    all_fids.append(fid_base)
    print('fid base', fid_base)
    
    rho_MLE_rot, best_phis, fid_rot = opt_virtualZ_MLE(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id, phis=phis)
    all_fids_rot.append(fid_rot)
    tomo_analysis.show_plot_rho_2d(rho_MLE_rot, rho_id, title=f'MLE with ZZ Correction, Phase Optimized (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]})', savetitle=None, cmax=0.5)
    print('fid rot', fid_rot)
    


In [ ]:
# print(all_n_tomo)
# print(all_n_calib)
# print(all_fids)
print('fidelity with optimized rotation\n', all_fids_rot)
print('saved files\n', saved_files)

In [ ]:
plt.plot(opt_lens, all_fids_rot, label='Q2 measured fidelity')

# decay_time = 10e3 # ns
# max_fid_times = np.linspace(0, max(opt_lens), 100)
# max_fids = np.exp(-max_fid_times/decay_time)
# plt.plot(max_fid_times, max_fids, '--', label=f'{decay_time/1e3} us decay')

# decay_time = 1e3 # ns
# max_fid_times = np.linspace(0, max(opt_lens), 100)
# max_fids = np.exp(-max_fid_times/decay_time)
# plt.plot(max_fid_times, max_fids, '--', label=f'{decay_time/1e3} us decay')

plt.xlabel(f'Length of 2Q swap in full protocol (ns)')
plt.ylabel(f'Q{sweep2Q_q} Fidelity')
plt.title(f'Q{sweep2Q_q}/Q1 parameter sweep from initial state {init_state}')
plt.legend()
plt.show()